# messy code ahead

This notebook contains the code that was developed for the thesis.

We combine AdaNet using a Bayesian optimizer from skopt to form a neural network efficiently.

However, since this code was not used in the final thesis the code might lack in quality and clarity.

### Import packages

In [ ]:
!pip3 install prosail
!pip3 install --upgrade scikit-learn scipy numpy adanet==0.9.0 dask distributed SALib tensorflow parameter-sherpa

In [ ]:
import numpy as np
import sklearn
import pandas as pd
import prosail as prosaillib
from scipy import stats
from scipy.stats import truncnorm
import adanet
from sklearn.preprocessing import normalize
#import tensorflow as tf
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
#tf.get_logger().setLevel('INFO')
import skopt

pd.set_option("display.max_rows", None)

In [ ]:
def biomass_from_height(height):
    #return (14.706*height - 12.094) / 10000
    return -0.0120942 + 0.0147056 * height

def biomass_from_lai_and_lma(lai, lma):
    return lai * lma

### Load random data

In [ ]:
df = pd.read_csv('./calculated_prosail_combined.csv', index_col=0).head(10_000)

In [ ]:
from sklearn.model_selection import train_test_split

calculated = df.to_numpy()
X = calculated[:, :9]
Y = calculated[:, 10:]
Y = np.array([ row[0]*row[1] for row in Y ])

x_train, x_test, y_train, y_test = train_test_split(X, Y, shuffle=False)

print(x_train[0], '->', y_train[0])

print('training samples', x_train.shape[0], 'test samples', x_test.shape[0])

### Fit the training set

In [ ]:
LOG_DIR = '/tmp/'

import shutil

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

print("running")

## Install ngrok binary.
#! rm -f ngrok-stable-linux-amd64.zip ngrok
#! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#! unzip ngrok-stable-linux-amd64.zip
#
## Delete old logs dir.
#shutil.rmtree(LOG_DIR, ignore_errors=True)
#
#print("Follow this link to open TensorBoard in a new tab.")
#get_ipython().system_raw('./ngrok http 6006 &')
#!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
from adanet.subnetwork import Generator
from skopt.space.space import Integer

adanet_dimensions = [
    Integer(low=1, high=100, name='dnn_layer_one'),
    Integer(low=-15, high=100, name='dnn_layer_two'),
    Integer(low=-15, high=100, name='dnn_layer_three'),
    #Integer(low=-15, high=200, name='dnn_layer_four'),
    #Integer(low=-15, high=100, name='dnn_layer_five')
]

opts = {} # iteration -> gp_object
info_points = { 0: [] } # iteration -> [(dnn_params, loss)]

class GPGenerator(Generator):
    def __init__(self):
        super()
        
    def generate_candidates(self, previous_ensemble, iteration_number, previous_ensemble_reports, all_reports, config):
        global next_iter, info_for_next_iter
        print('iteration', iteration_number)
        
        if iteration_number+1 not in info_points:
            info_points[iteration_number+1] = []
        
        opt = opts.get(iteration_number)
        if opt is None:
            n_initial_points = 9 if iteration_number == 0 else 0
            opts[iteration_number] = skopt.optimizer.Optimizer(dimensions=adanet_dimensions, base_estimator="GP", initial_point_generator="sobol", n_initial_points=n_initial_points)
            opt = opts[iteration_number]
            
            #for (params, loss) in info_points[iteration_number]:
            for report in previous_ensemble_reports:
                if not report.name.startswith("dnn_"):
                    continue
                loss = report.metrics['loss']
                params = [int(x) for x in report.name.split('_')[1:]]
                print('telling', params, loss)
                opt.tell(params, loss)
        
        for report in previous_ensemble_reports:
            if not report.name.startswith("dnn_"):
                continue
                
            loss = report.metrics['loss']
            params = [int(x) for x in report.name.split('_')[1:]]
            print('storing', params, loss)
            info_points[iteration_number].append((params, loss))
        
        def conv(name, x):
            return adanet.autoensemble.common._BuilderFromSubestimator(
                name,
                adanet.autoensemble.common._convert_to_subestimator(x),
                logits_fn=adanet.autoensemble.common._default_logits,
                last_layer_fn=None,
                config=config)
        
        res = [conv("linear", tf.estimator.LinearEstimator(
                head=head,
                feature_columns=feature_columns,
                config=config))]
        
        points = opt.ask(n_points=9)
        used = set()
        for point in points:
            #name = "dnn_{}_{}_{}_{}_{}".format(point[0], point[1], point[2], point[3], point[4])
            name = "dnn_{}_{}_{}".format(point[0], point[1], point[2])
            if name in used:
                continue
            used.add(name)
            
            new_point = [point[0]]
            for i in range(1, len(point)):
                new_point.append(new_point[-1] - point[i])
            point = new_point
            
            print("adding", name, end=" ")
            
            hidden_units = []
            for x in point:
                if x <= 0:
                    break
                hidden_units.append(x)
            print(hidden_units)
                
            item = tf.estimator.DNNEstimator(
                    head=head,
                    feature_columns=feature_columns,
                    config=config,
                    hidden_units=hidden_units)
            
            res.append(conv(name, item))
        return res

In [ ]:
%%time

FEATURES_KEY = "x"

#col_names = ['solar_zenith', 'solar_azimuth', 'sensor_zenith', 'sensor_azimuth', 'B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B8']
col_names = [ "x" ]
feature_columns = [ tf.feature_column.numeric_column(x, shape=[11,1]) for x in col_names ]

head = tf.estimator.RegressionHead(label_dimension=1)
generator = GPGenerator()

def f(config):
    print('config', config)
    return {
        "linear":
            tf.estimator.LinearEstimator(
                head=head,
                feature_columns=feature_columns,
                config=config),
        "dnn":
            tf.estimator.DNNEstimator(
                head=head,
                feature_columns=feature_columns,
                config=config,
                hidden_units=[25]),
        "dnn2_50-25":
            tf.estimator.DNNEstimator(
                head=head,
                feature_columns=feature_columns,
                config=config,
                hidden_units=[50, 25]),
        "dnn2_100-25":
            tf.estimator.DNNEstimator(
                head=head,
                feature_columns=feature_columns,
                config=config,
                hidden_units=[100, 25]),
        "dnn3":
            tf.estimator.DNNEstimator(
                head=head,
                feature_columns=feature_columns,
                config=config,
                hidden_units=[200, 100, 25]),
    }

def input_fn_train():
    dataset = tf.compat.v1.data.Dataset.from_tensor_slices(({
        FEATURES_KEY: x_train
    }, y_train))
    #dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset = dataset.batch(32)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

def input_fn_eval():
    dataset = tf.compat.v1.data.Dataset.from_tensor_slices(({
        FEATURES_KEY: x_test
    }, y_test))
    dataset = dataset.batch(32)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

estimator = adanet.Estimator(
    head=head,
    #candidate_pool=[],
    subnetwork_generator=generator,
    max_iteration_steps=60,
    report_materializer=adanet.ReportMaterializer(input_fn_eval),
    max_iterations=15
)

estimator.train(input_fn=input_fn_train)
metrics = estimator.evaluate(input_fn=input_fn_eval, steps=10)
print(metrics)

#### ... and evaluate the result on other artificial data

In [ ]:
def input_fn_eval():
    dataset = tf.compat.v1.data.Dataset.from_tensor_slices(({
        FEATURES_KEY: x_test
    }, y_test))
    #dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset = dataset.batch(32)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

import sklearn.metrics

#predicted = list(estimator.predict(input_fn=input_fn_eval))
#predicted = np.array([ x['predictions'] for x in estimator.predict(input_fn=input_fn_eval) ])
predicted = np.array([ x['predictions'] for x in estimator.predict(input_fn=input_fn_eval) ])
print(predicted)
print("score over test set using double var", sklearn.metrics.mean_absolute_percentage_error(y_test, predicted))

In [ ]:
yhat = predicted
yhat.shape

In [ ]:
from matplotlib import pyplot

pyplot.scatter(y_test, yhat)
pyplot.show()

"""
pyplot.scatter(y_test[:, 0], yhat[:, 0])
pyplot.show()

pyplot.scatter(y_test[:, 1], yhat[:, 1])
pyplot.show()
"""

## Let's now test it on a real dataset

### Load in the dataset

In [ ]:
items = pd.read_csv('/content/drive/MyDrive/colabfiles/data_with_angles.csv')
etm_rows = items[items['SensorType'] == 'ETM+']
etm_rows = etm_rows.dropna(subset=['h_grass', 'c_grass'])

bands = etm_rows.filter(regex='B\d$') / 10_000
grass_info = etm_rows.filter(regex='_grass$')
angles = etm_rows.filter(regex='B\d_.+$')

items = grass_info.join(bands).join(angles)
items

In [ ]:
def mean(df):
    x = df.to_numpy()
    return np.nanmean(x, axis=1)

def get_angles(df):
    sensor_azimuth = mean(df.filter(regex='B\d_sensor_azimuth'))
    sensor_zenith = mean(df.filter(regex='B\d_sensor_zenith'))
    solar_azimuth = mean(df.filter(regex='B\d_solar_azimuth'))
    solar_zenith = mean(df.filter(regex='B\d_solar_zenith'))
    return np.column_stack((solar_zenith, solar_azimuth, sensor_zenith, sensor_azimuth))

x_test_bands = items.filter(regex='B\d$').to_numpy()
x_test_bands = normalize(x_test_bands, norm='max', axis=0)
x_test_angles = get_angles(items)
x_test_angles = normalize(x_test_angles, norm='max')
x_test = np.column_stack((x_test_angles, x_test_bands))

y_test = np.array([ biomass_from_height(x[0]) for x in items[['h_grass']].to_numpy() ])
#y_test = [ x[0] for x in items[['h_grass']].to_numpy() ]
#y_test = [ y if y >= 0 else 0 for y in y_test ]

#y_test = items[['h_grass', 'c_grass']].to_numpy()
print(x_test[0], '->', y_test[0])

In [ ]:
params.describe()

In [ ]:
x_test_prosail = np.array([ prosail_to_etm(etm_rsr, prosail(row)) for (_,row) in test_params.iterrows() ])
x_test_prosail = normalize(x_test_prosail, norm='max', axis=0)
pd.DataFrame(x_test_prosail).describe()

In [ ]:
pd.DataFrame(x_test_bands).describe()

In [ ]:
predicted = np.array([ biomass_from_lai_and_lma(row[0], row[1]) for row in cls.predict(x_test)])
print("score over test set using single var", sklearn.metrics.mean_absolute_percentage_error(y_test, predicted))

In [ ]:
from matplotlib import pyplot

yhat = predicted

pyplot.scatter(y_test, yhat)
pyplot.show()

In [ ]:
sgpitats.describe(np.array(x_test)[:1000] - np.array(x_train)[:1000])

In [ ]:
stats.describe(np.array(y_test)[:1000] - np.array(y_train)[:1000])

In [ ]:
res = gpr.predict(x_test)
heights = [ 68.0013*m + 0.822421 for m in res ]
heights

In [ ]:
np.abs(np.array(y_test) - np.array(heights))

In [ ]:
diff = np.abs(items['h_grass'].to_numpy() - np.array(heights))
print(stats.describe(diff))
print(diff)

In [ ]:
items

In [ ]:
x_test

In [ ]:
y_test

In [ ]:
cls.predict(x_test)

In [ ]:
test_params[:, -5:-1][0]

In [ ]:
import SALib.sample.latin

SALib.sample.latin.sample({ 'num_vars': 1, 'names': ['x1'], 'bounds': [[-10, 10]] }, 10)